## Кривоногов Н.В., NLP, практическое задание № 14

1.  Дообучить берт на задачу NER
2.  Дообучить GPT на генерацию текста

In [1]:
! pip install datasets transformers seqeval

In [2]:
model_checkpoint = "cointegrated/rubert-tiny"
batch_size = 16

## Loading the dataset

In [3]:
from datasets import load_dataset, load_metric

In [4]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
!pip install corus razdel

--2023-09-21 19:00:20--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2023-09-21 19:00:20--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json.2’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.008s  

2023-09-21 19:00:20 (205 MB/s) - ‘rudrec_annotated.json.2’ saved [1773014/1773014]



In [5]:
from corus import load_rudrec
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


Посмотрю, какие сущности есть: лекарства, форма лекарств, класс лекарств, показания к применению, побочки, и прочие болезни/симптомы.

https://arxiv.org/abs/2004.03659

* **DRUGNAME** Mentions of the brand name of a drug or product
ingredients/active compounds.
* **DRUGCLASS** Mentions of drug classes such as anti-inflammatory or
cardiovascular.
* **DRUGFORM** Mentions of routes of administration such as tablet
or liquid that describe the physical form in which
medication will be delivered into patient’s organism.
* **DI** Any indication/symptom that specifies the reason for
taking/prescribing the drug.
* **ADR** Mentions of untoward medical events that occur as a
consequence of drug intake and are not associated with
treated symptoms.
* **FINDING** Any DI or ADR that was not directly experienced by the
reporting patient or his/her family members, or related to
medical history/drug label, or any disease entities if the
annotator is not clear about type

In [7]:
from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))

DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


In [8]:
drugs[0].text

'нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n'

Напишу функцию, перекладывающую разметку сущностей на уровень слов. Буду использовать [IOB](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging))-нотацию, чтобы разделять несколько сущностей одного типа, идущих подряд.

In [9]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [10]:
print(extract_labels(drugs[0]))

{'tokens': ['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [11]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.1, random_state=1)

Пример данных

In [12]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
1123,"[Этот, сироп, может, применяться, как, для, профилактики, ,, так, и, для, раннего, лечения, ОРВИ, и, Гриппа, .]","[O, B-Drugform, O, O, O, O, B-DI, O, O, O, O, O, O, B-DI, O, B-DI, O]"
2380,"[Будьте, осторожны, !!]","[O, O, O]"
2426,"[Приняла, я, всего, три, таблетки, ,, как, начался, ужасный, зуд, ,, а, затем, пошла, мелкая, сыпь, по, всему, телу, .]","[O, O, O, O, B-Drugform, O, O, O, O, B-ADR, O, O, O, O, O, B-ADR, I-ADR, I-ADR, I-ADR, O]"


Соберу все виды меток в список.

In [13]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

Сложу данные в объект [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), нативный для huggingface.

In [14]:
from datasets import Dataset, DatasetDict

In [15]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 4328
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 481
    })
})

## Preprocessing the data

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [17]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 7440, 996, 6301, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
example = ner_train[5]
print(example["tokens"])

['Этим', 'средством', 'является', '"', 'Амизон', '"', ',', 'которое', 'хорошо', 'прорекламировано', ',', 'однако', 'такого', 'эффекта', 'как', 'говорят', 'нет', '.']


In [20]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Эти', '##м', 'средство', '##м', 'является', '"', 'А', '##ми', '##зон', '"', ',', 'которое', 'хорошо', 'про', '##рек', '##лами', '##ровано', ',', 'однако', 'такого', 'э', '##ффект', '##а', 'как', 'говорят', 'нет', '.', '[SEP]']


Чтобы перейти с уровня слов на уровень subword tokens, нужно ещё раз предобработать тексты.

In [21]:
len(example["tags"]), len(tokenized_input["input_ids"])

(18, 29)

In [22]:
print(tokenized_input.word_ids())

[None, 0, 0, 1, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 9, 9, 9, 10, 11, 12, 13, 13, 13, 14, 15, 16, 17, None]


In [23]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

29 29


In [24]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [25]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 3130, 3374, 23324, 871, 314, 1556, 14068, 16902, 1029, 6899, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [26]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/4328 [00:00<?, ? examples/s]

Map:   0%|          | 0/481 [00:00<?, ? examples/s]

## Fine-tuning the model

In [27]:
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [28]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
! pip install -U accelerate
! pip install -U transformers

In [30]:
import accelerate

accelerate.__version__

'0.23.0'

In [31]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [32]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [33]:
metric = load_metric("seqeval")

<ipython-input-33-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [34]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

In [35]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [36]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [37]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 2.6584620475769043,
 'eval_precision': 0.015241952675335749,
 'eval_recall': 0.08708891595615104,
 'eval_f1': 0.025943396226415096,
 'eval_accuracy': 0.048060849214309594,
 'eval_runtime': 4.4506,
 'eval_samples_per_second': 108.074,
 'eval_steps_per_second': 6.965}

В начале обучения заморожу все параметры в модели, кроме последнего слоя, и посмотрю, насколько хорошо она обучится.

In [38]:
for param in model.bert.parameters():
    param.requires_grad = False

In [39]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)

classifier.weight
Parameter containing:
tensor([[ 1.3071e-02,  9.7536e-03,  1.3512e-02,  ..., -2.1082e-02,
          5.4800e-02, -4.3229e-03],
        [-1.9266e-02, -4.9144e-02, -5.1117e-02,  ...,  4.3882e-02,
          1.2737e-02,  2.8279e-02],
        [ 2.9891e-03,  4.1530e-03,  1.8372e-02,  ..., -1.8494e-02,
          1.3712e-02, -1.7718e-02],
        ...,
        [-7.7672e-03, -2.7181e-02, -1.8086e-02,  ...,  8.8437e-03,
         -1.0436e-02,  2.3519e-03],
        [ 1.4233e-06, -2.4850e-02,  1.4642e-02,  ..., -1.5708e-02,
         -1.0438e-02,  9.7797e-03],
        [-3.4851e-02,  5.7925e-03,  2.4615e-02,  ..., -5.5635e-03,
         -2.0130e-04,  2.0760e-02]], device='cuda:0', requires_grad=True)
classifier.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)


In [40]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,2.029289,0.025358,0.042022,0.031630,0.627633
2,2.106200,1.553912,0.029703,0.001827,0.003442,0.811685
3,2.106200,1.233240,0.000000,0.000000,0.000000,0.818121
4,1.297500,1.039616,0.000000,0.000000,0.000000,0.818288
5,1.297500,0.930293,0.000000,0.000000,0.000000,0.818372
6,0.928600,0.870161,1.000000,0.000609,0.001217,0.818455
7,0.928600,0.836648,1.000000,0.000609,0.001217,0.818455
8,0.808900,0.818012,1.000000,0.001827,0.003647,0.818623
9,0.808900,0.808619,1.000000,0.002436,0.004860,0.818706
10,0.757400,0.805705,1.000000,0.002436,0.004860,0.818706


TrainOutput(global_step=2710, training_loss=1.1463661334611392, metrics={'train_runtime': 54.6601, 'train_samples_per_second': 791.803, 'train_steps_per_second': 49.579, 'total_flos': 40083054698880.0, 'train_loss': 1.1463661334611392, 'epoch': 10.0})

Модель недообучилась: похоже, что нужно обучить больше слоёв. Разморожу их все (но, воможно, более правильно было бы разморозить лишь несколько верхних), и поучимся ещё эпох 20.

In [42]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [43]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [44]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [45]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.606948,0.646655,0.229598,0.338876,0.846540
2,0.562600,0.536534,0.582503,0.320341,0.413360,0.856068
3,0.562600,0.492387,0.586301,0.390987,0.469127,0.864594
4,0.448100,0.466244,0.582689,0.459196,0.513624,0.872534
5,0.448100,0.439719,0.609195,0.516443,0.558998,0.881227
6,0.387200,0.422270,0.601474,0.546894,0.572887,0.884487
7,0.387200,0.410346,0.632568,0.553593,0.590451,0.888081
8,0.348700,0.400206,0.645543,0.564555,0.602339,0.891174
9,0.348700,0.389746,0.637216,0.579781,0.607143,0.893347
10,0.326800,0.381446,0.621317,0.603532,0.612295,0.893932


TrainOutput(global_step=5420, training_loss=0.34085366787505766, metrics={'train_runtime': 144.4313, 'train_samples_per_second': 599.316, 'train_steps_per_second': 37.526, 'total_flos': 80264410784640.0, 'train_loss': 0.34085366787505766, 'epoch': 20.0})

In [46]:
trainer.evaluate()

{'eval_loss': 0.3527190089225769,
 'eval_precision': 0.6302623550945698,
 'eval_recall': 0.6291108404384896,
 'eval_f1': 0.6296860713197195,
 'eval_accuracy': 0.8992811768639251,
 'eval_runtime': 0.9369,
 'eval_samples_per_second': 513.409,
 'eval_steps_per_second': 33.089,
 'epoch': 20.0}

In [47]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'ADR': {'precision': 0.3933649289099526,
  'recall': 0.3132075471698113,
  'f1': 0.3487394957983193,
  'number': 265},
 'DI': {'precision': 0.4430379746835443,
  'recall': 0.5372807017543859,
  'f1': 0.48562933597621405,
  'number': 456},
 'Drugclass': {'precision': 0.7839506172839507,
  'recall': 0.7888198757763976,
  'f1': 0.7863777089783281,
  'number': 161},
 'Drugform': {'precision': 0.8390804597701149,
  'recall': 0.8202247191011236,
  'f1': 0.8295454545454546,
  'number': 267},
 'Drugname': {'precision': 0.7991071428571429,
  'recall': 0.8927680798004988,
  'f1': 0.8433451118963488,
  'number': 401},
 'Finding': {'precision': 0.25,
  'recall': 0.010869565217391304,
  'f1': 0.020833333333333332,
  'number': 92},
 'overall_precision': 0.6302623550945698,
 'overall_recall': 0.6291108404384896,
 'overall_f1': 0.6296860713197195,
 'overall_accuracy': 0.8992811768639251}

In [48]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [49]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ADR,B-DI,B-Drugclass,B-Drugform,B-Drugname,B-Finding,I-ADR,I-DI,I-Drugclass,I-Drugform,I-Drugname,I-Finding
O,9619,16,67,12,20,34,1,10,12,0,0,0,0
B-ADR,84,89,73,8,0,0,2,8,1,0,0,0,0
B-DI,154,21,262,1,11,4,0,3,0,0,0,0,0
B-Drugclass,22,0,9,127,0,3,0,0,0,0,0,0,0
B-Drugform,34,0,4,1,220,8,0,0,0,0,0,0,0
B-Drugname,19,0,11,1,2,367,0,0,1,0,0,0,0
B-Finding,22,26,29,5,5,3,1,0,1,0,0,0,0
I-ADR,108,10,18,0,1,0,0,45,20,0,0,0,0
I-DI,143,19,48,0,0,1,0,12,29,0,0,0,0
I-Drugclass,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

# Применение модели

In [51]:
import torch

In [52]:
text = ' '.join(ner_train[8]['tokens'])
text = ' '.join(ner_test[4]['tokens'])
text

'Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .'

In [53]:
import torch

In [54]:
tokens = tokenizer(text, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}

with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

torch.Size([1, 29, 13])

In [55]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
О               O         
##хо            O         
##тно           O         
при             O         
##мен           O         
##я             O         
##ю             O         
его             O         
при             O         
борьбе          O         
с               O         
нас             B-DI      
##мор           B-DI      
##ком           B-DI      
,               O         
что             O         
в               O         
м               O         
##ое            O         
##м             O         
случае          O         
я               O         
##вление        O         
очень           O         
часто           O         
##е             O         
.               O         
[SEP]           O         


Более простое применение модели: пайплайн от huggingface

In [56]:
from transformers import pipeline

In [57]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [58]:
print(text)
print(pipe(text))

Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .
[{'entity_group': 'DI', 'score': 0.79165894, 'word': 'насморком', 'start': 33, 'end': 42}]


In [59]:
!pip install transformers

In [60]:
!ls

ner	      rudrec_annotated.json    rudrec_annotated.json.2
ner_bert.bin  rudrec_annotated.json.1  sample_data


https://drive.google.com/file/d/14XXOmUlKFk9n9QT82wxLS6Ul11E9jyDt/view?usp=sharing

In [65]:
!gdown 14XXOmUlKFk9n9QT82wxLS6Ul11E9jyDt

Downloading...
From: https://drive.google.com/uc?id=14XXOmUlKFk9n9QT82wxLS6Ul11E9jyDt
To: /content/archive.zip
100% 16.7M/16.7M [00:00<00:00, 26.9MB/s]


In [66]:
!unzip -o '/content/archive.zip'

Archive:  /content/archive.zip
  inflating: all_recepies_inter.csv  


In [61]:
import pandas as pd

In [67]:
df_rec = pd.read_csv('/content/all_recepies_inter.csv', sep='\t')

In [68]:
df_rec[:4]

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными огурцами,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, {'Соленые огурцы': 0.4, 'unit': 'шт.'}, {'Морковь': 0.2, 'unit': 'шт.'}, {'Лук': 0.2, 'unit': 'шт.'}, {'Чеснок': 0.2, 'unit': 'зубч.'}, {'Стебель сельдерея': 1, 'unit': 'шт.'}, {'Картошка': 0.4, 'unit': 'шт.'}, {'Бульон': 0.5, 'unit': 'л'}, {'Томатна...","варка,жарка","Подготовить указанные ингредиенты для приготовления рассольника с перловой крупой. Мясной бульон сварить заранее из говядины или из курицы, также можно сварить и вегетарианский суп - на воде. Обычно я рассольник варю без томатной пасты, но тут для разнообразия решила добавить - это по желанию. И...",первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-perlovkoi-klassicheskii,"[{'product_id': 4253, 'name_source': 'Перловая крупа', 'unit_id': 5, 'unit_source': 'стак. (200 мл)', 'value_in_recipe': 0.1, 'weight_source': 18.0, 'weight_handle': 0}, {'product_id': 4011, 'name_source': 'Огурец соленый', 'unit_id': 1, 'unit_source': 'шт.', 'value_in_recipe': 0.4, 'weight_sour..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'}, {'Картошка': 30.0, 'unit': 'гр'}, {'Лук репчатый': 10.0, 'unit': 'гр'}, {'Морковь': 10.0, 'unit': 'гр'}, {'Чеснок': 0.2, 'unit': 'зубч.'}, {'Лавровый лист': 0.2, 'unit': 'шт.'}, {'Оливковое масло': 0.2, 'unit': 'стол.л.'}, {'Поваренная соль': 'по вк...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морковь и чеснок, овощи можно особо не мельчить потому как все будем превращать в пюре\r\nПассеруем морковь и лук на оливковом масле на сильном огне буквально пару минут, добавляем чеснок, перемешиваем и снимаем с огня\r\nПока овощи обжариваются, крупно н...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-belokochanoi-kapusty,"[{'product_id': 2286, 'name_source': 'Капуста белокочанная', 'unit_id': 4, 'unit_source': 'гр', 'value_in_recipe': 50.0, 'weight_source': 50.0, 'weight_handle': 0}, {'product_id': 2334, 'name_source': 'Картофель', 'unit_id': 4, 'unit_source': 'гр', 'value_in_recipe': 30.0, 'weight_source': 30.0,..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'Мак пищевой': 0.5, 'unit': 'стол.л.'}, {'Морковь': 0.2, 'unit': 'шт.'}, {'Лук репчатый': 0.3, 'unit': 'шт.'}, {'Репа': 0.2, 'unit': 'шт.'}, {'Растительное масло': 0.3, 'unit': 'стол.л.'}, {'Чеснок': 0.3, 'unit': 'зубч.'}, {'Мука': 0.2, 'unit': 'стол....","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момент приготовления, но я прекрасно обошлась и без нее.Возьмем овощи: морковку, репу и корень петрушки, обмоем, и очистим. Порежем овощи мелкими кубиками. То же самое проделаем с луком.На разогретой сковороде с растительным маслом спассируем подготовленн...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста квашеная', 'unit_id': 4, 'unit_source': 'гр', 'value_in_recipe': 116.7, 'weight_source': 116.7, 'weight_handle': 0}, {'product_id': 3339, 'name_source': 'Мак, семена', 'unit_id': 2, 'unit_source': 'стол.л.', 'value_in_recipe': 0.5, 'weight_source': 9.0,..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': 0.2, 'unit': 'шт.'}, {'Черный хлеб': 83.3, 'unit': 'гр'}, {'Чеснок': 0.5, 'unit': 'зубч.'}, {'Зелёный лук': 'по вкусу', 'unit': 'по вкусу'}, {'Зелень кинзы': 3.3, 'unit': 'гр'}, {'Соль': 'по вкусу', 'unit': 'по вкусу'}]",сырое,"\r\nНачинаем мы приготовление тюри с того, что с заранее подготовленного черного хлеба срезаем корки, нарезаем их небольшими ломтиками. После этого кладем наши кусочки хлеба на противень и подсушиваем в умеренно разогретом духовом шкафу в течение примерно 10 минут. Затем каждый подсушенный кусоч...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name

In [69]:
print(df_rec.loc[0, 'Инструкции'])

Подготовить указанные ингредиенты для приготовления рассольника с перловой крупой. Мясной бульон сварить заранее из говядины или из курицы, также можно сварить и вегетарианский суп - на воде. Обычно я рассольник варю без томатной пасты, но тут для разнообразия решила добавить - это по желанию. Из специй соль, чёрный перец горошком, душистый перец.
Перловую крупу промыть до чистой воды.
В горячий бульон добавить промытую перловку и варить на среднем огне.
Для рассольника лучше брать кислые, очень солёные огурцы. Если же огурцы обычные, то рекомендуется в рассольник добавлять из сам рассол от огурцов. Солёные огурцы достать из рассола и натереть на крупной тёрке.
Картофель помыть, обсушить, очистить. Нарезать кубиками. Пока очередь картофеля не подошла, положить его в воду.
Морковь, лук, чеснок очистить. Морковь натереть на крупной тёрке, лук, сельдерей, чеснок мелко порезать.
Обжарить в масле овощи, добавив томатной пасты. Томатная паста по желанию.
Минут через двадцать добавить к перло

In [70]:
print(df_rec.loc[10, 'Инструкции'])

Приготовим ингредиенты по списку. Поскольку мы используем свежую капусту, а не квашенную, то должен быть какой-то продукт, который создаст кислый вкус, отличающий его от других супов. Это могут быть антоновские яблоки, щавель или соленые грибы. Я использовала последний из них.
Морковь моем, чистим и натираем на средней терке.
Репчатый лук чистим от шелухи и мелко нарезаем ножом.
Белокочанную капусту очищаем от верхних сухих листьев и мелко шинкуем ножом.
Картофель моем, чистим и нарезаем небольшими кубиками или соломкой, снова промываем, чтобы удалить крахмал и сделать бульон прозрачным. Картофель сделает суп гуще и сытнее.
Грибы можно использовать любые: шампиньоны или лесные, свежие, маринованные, соленые. Я использовала соленые рыжики, с их добавлением щи получились ароматными и с пикантной легкой остринкой и кислинкой, именно это является отличительной чертой супа - щей. Соленые грибы промываем, даем воде стечь, если грибочки крупные нарезаем их на маленькие кубики. Если используют

In [71]:
df_rec.shape

(27884, 10)

In [72]:
data = df_rec.loc[:5000, 'Инструкции']

In [73]:
import re
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [74]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [75]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 4250
Test dataset length: 751


In [76]:
from transformers import AutoTokenizer
#sberbank-ai/rugpt3large_based_on_gpt2
#sberbank-ai/rugpt3medium_based_on_gpt2
#sberbank-ai/rugpt3small_based_on_gpt2

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [77]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [78]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [79]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [80]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [81]:
trainer.train()

Step,Training Loss
500,2.528600
1000,2.440000
1500,2.376400
2000,2.236900
2500,2.160400
3000,2.147000
3500,2.093300
4000,1.975500
4500,1.974000
5000,1.968400


TrainOutput(global_step=5025, training_loss=2.1888466807503013, metrics={'train_runtime': 1066.7371, 'train_samples_per_second': 18.84, 'train_steps_per_second': 4.711, 'total_flos': 1312796491776000.0, 'train_loss': 2.1888466807503013, 'epoch': 3.0})

In [82]:
trainer.save_model()

In [83]:
tokenizer.save_pretrained('gpt_chf')

('gpt_chf/tokenizer_config.json',
 'gpt_chf/special_tokens_map.json',
 'gpt_chf/vocab.json',
 'gpt_chf/merges.txt',
 'gpt_chf/added_tokens.json',
 'gpt_chf/tokenizer.json')

In [84]:
tokenizer.save_pretrained('gpt_chf')
model.save_pretrained('model_gpt_chf')

In [85]:
tokenizer = AutoTokenizer.from_pretrained("gpt_chf")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_chf")

In [86]:
prefix = "режем сало на тонкие кусочки "

In [87]:
tokens = tokenizer(prefix, return_tensors='pt')
#tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [88]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens,
    #end_token=end_token_id,
    do_sample=False,
    max_length=size+50,
    repetition_penalty=5.,
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


режем сало на тонкие кусочки  2. В сковороде разогреваем растительное масло и обжариваем лук до золотистого цвета, затем добавляем нарезанный полукольцами шампиньон и жарим все вместе еще 3–4 минуты. Добавляем томатную па


In [89]:
prefix = "разбиваем 13 яиц и взбалтываем их "

In [90]:
tokens = tokenizer(prefix, return_tensors='pt')
#tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [91]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens,
    #end_token=end_token_id,
    do_sample=False,
    max_length=size+50,
    repetition_penalty=5.,
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


разбиваем 13 яиц и взбалтываем их  5. Выкладываем на противень, смазанный растительным маслом, и запекаем в разогретой до 180 градусов духовке 15-20 минут (зависит от конкретной духовки).  6. Готовый суп разливаем по тарел


In [92]:
prefix = "наливаем рюмку водки "

In [93]:
tokens = tokenizer(prefix, return_tensors='pt')
#tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [94]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens,
    #end_token=end_token_id,
    do_sample=False,
    max_length=size+50,
    repetition_penalty=5.,
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


наливаем рюмку водки  и ставим в духовку, разогретую до 180 градусов на 40 минут.  1. Разогрейте оливковое масло в большой кастрюле на среднем огне. Добавьте мелко нарезанный лук-шалот и обжарьте 5 минут
